In [16]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

#### 변경 셀 사이즈 및 KATECH 좌표 원점 ####
#### x_base, y_base는 지역에 상관없이 고정값 ####
cell_size = 100
x_base,y_base = 400000,600000

def cv_cell(x,y):  ### 셀 사이즈 변경 함수 정의 ###
    a = ((x - x_base)//cell_size)*cell_size + (cell_size//2 -25) + x_base
    b = ((y - y_base)//cell_size)*cell_size + (cell_size//2 -25) + y_base
    c = (a//100)*10000 + (b//100)
    return(c,a,b)

In [19]:
path = f"C:\\Users\\dudtj\\busan_moving_person\\data\\opt_data" #원천데이터 파일위치
os.chdir(path)
file_list = os.listdir(path)
f_list = [file for file in file_list if file.endswith(".csv")]
print(f_list)

['pr_opt_analysis.csv']


In [25]:
test_df = pd.read_csv(f_list[0], encoding="UTF-8").dropna(axis=0)
test_df.to_csv('pr_opt_analysis.csv', encoding='UTF-8')

In [23]:
test_df

,id,x,y,timezn_cd,m_201901,f_201901,m_201902,f_201902,m_201903,f_201903,...,m_201908,f_201908,m_201909,f_201909,m_201910,f_201910,m_201911,f_201911,m_201912,f_201912
0,95139666,492452,284793,0,56.08,48.81,50.52,40.86,57.27,48.81,...,67.51,65.70,63.36,63.28,64.73,64.08,47.26,57.11,44.85,45.23
1,95139666,492452,284793,1,65.96,57.05,56.17,44.98,66.17,56.86,...,84.25,78.27,80.40,79.53,81.67,78.08,63.00,73.17,58.78,57.79
2,95139666,492452,284793,2,71.10,61.11,59.77,49.77,69.11,58.39,...,86.49,80.42,80.78,80.51,87.59,80.69,69.59,75.35,61.90,63.65
3,95139666,492452,284793,3,77.09,64.61,65.44,51.49,70.08,62.30,...,90.66,83.50,85.86,84.17,92.70,79.59,72.35,76.49,64.55,67.74
4,95139666,492452,284793,4,80.41,68.43,68.36,54.21,71.36,64.30,...,92.30,85.50,89.09,84.30,94.85,82.07,75.88,75.31,67.02,71.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198466,96687672,498802,284943,19,53.65,53.18,63.91,58.88,64.18,61.43,...,116.79,131.80,100.78,112.15,95.25,101.87,97.38,100.33,67.67,65.77
198467,96687672,498802,284943,20,67.61,63.16,84.97,72.16,90.93,83.44,...,200.49,206.87,146.32,152.46,139.86,137.00,122.57,122.72,84.48,79.27
198468,96687672,498802,284943,21,73.51,67.31,89.72,75.65,93.63,83.57,...,235.91,245.84,158.26,170.67,145.27,146.46,132.11,127.41,93.58,83.24
198469,96687672,498802,284943,22,62.00,59.32,74.71,64.96,80.57,76.59,...,215.15,220.46,147.04,155.34,131.13,133.57,111.54,103.61,87.23,78.20


In [24]:
# 칼럼 명 < 원천 데이터 확인 필요 > 
col_name = ['id', 'x', 'y', 'm', 'f', 'total']

# 칼럼 수 Test & Check
test_df = pd.read_csv(f_list[0], encoding="UTF-8")
print("col_name 칼럼 수 확인 : ", len(col_name))
print("raw_data 칼럼 수 확인 : ", len(test_df.columns))

col_name 칼럼 수 확인 :  6
raw_data 칼럼 수 확인 :  28


In [7]:
test_df.columns

Index(['id', 'x', 'y', 'm', 'f', 'total'], dtype='object')

In [11]:
for filename in tqdm(f_list):
    print(filename)

##########################################################

    chunksize = 1000  ### 1번에 처리할 행의 갯수 정의 ##
    
##########################################################    
    
    cnt_list=[]
    for cnt, celldf in tqdm(enumerate(pd.read_csv(filename, header = 0, names=col_name, encoding = "UTF-8", chunksize=chunksize))):
        print(cnt, celldf.shape)
         
        a=[]
        cellar = celldf[['x','y']].values
        for rows in cellar:
            a.append(cv_cell(rows[0], rows[1]))
        
        celldf[['id','x','y']] = a
        celldf = celldf.groupby(['id', 'x', 'y'], as_index = False).sum()
        columns = ['id', 'x', 'y', 'm', 'f', 'total']
        celldf = celldf[columns]
        
        cnt_filename = filename[:-4]+str(cnt)+filename[-4:]
        cnt_list.append(cnt_filename)
        celldf.to_csv(cnt_filename, encoding = "UTF-8" ,index = False)
    
    celldf = pd.DataFrame(columns=columns)
    print('### 파일을 통합합니다. ###')
    
    for filename in cnt_list:
        adf = pd.read_csv(filename, header = 0, encoding = "UTF-8")
        celldf = pd.concat([celldf,adf])
        os.remove(filename)
        
    celldf = celldf.groupby(['id', 'x', 'y'], as_index = False).sum()
    celldf = celldf[columns]
    
    mg_filename = './CV100_'+filename[:-5]+'.csv'
    print(mg_filename)
    celldf.to_csv(mg_filename, encoding = "UTF-8" ,index = False)

  0%|          | 0/1 [00:00<?, ?it/s]

opt_analysis.csv


0it [00:00, ?it/s]

0 (1000, 6)
1 (1000, 6)
2 (1000, 6)
3 (1000, 6)
4 (1000, 6)
5 (1000, 6)
6 (1000, 6)
7 (1000, 6)
8 (1000, 6)
9 (1000, 6)
10 (239, 6)
### 파일을 통합합니다. ###
./CV100_opt_analysis1.csv
